In [1]:
import csv
import math
import random


### Getting csv data and converting into normalizing data frame

In [16]:
xDistanceToTarget=[]
yDistanceToTarget =[]
velocityX=[]
velocityY=[]

xDistanceToTargetNormalised=""
yDistanceToTargetNormalised =""
velocityXNormalised=""
velocityYNormalised=""

dataFrame =""
newDataFrame = ""

def normaliseDataSet(list):
    newNormalisedDataSet=[]
    for value in list:
        data = (value - min(list))/(max(list)-min(list))
        newNormalisedDataSet.append(data)
    return newNormalisedDataSet 


with open("ce889_dataCollection2.csv","r") as file:
    dataFrame = csv.reader(file)
    data = [row for row in dataFrame]
    for value in data:
        xDistanceToTarget.append(float(value[0]))
        yDistanceToTarget.append(float(value[1]))
        velocityX.append(float(value[2]))
        velocityY.append(float(value[3]))
        
xDistanceToTargetNormalised = normaliseDataSet(xDistanceToTarget)
yDistanceToTargetNormalised = normaliseDataSet(yDistanceToTarget)
velocityXNormalised = normaliseDataSet(velocityX)
velocityYNormalised = normaliseDataSet(velocityY)

minVelX = min(velocityX)
maxVelX=max(velocityX)
minVelY=min(velocityY)
maxVelY=max(velocityY)

dfWithoutNormalise = list(zip(xDistanceToTarget,yDistanceToTarget,velocityX,velocityY))  
newDataFrame = list(zip(normaliseDataSet(xDistanceToTarget),normaliseDataSet(yDistanceToTarget),normaliseDataSet(velocityX),normaliseDataSet(velocityY)))



### Neuron Class /Architecture 
Contains FeedForward Backpropogation method

In [17]:
class NeuronClass:
        
    def __init__(self,inputNodes,HLNeurons,OPLayerNodes,lambdaValue,learningRate,momentumRate,dataFrame):
        super().__init__()
        self.inputNodes = inputNodes
        self.HLNeurons = HLNeurons
        self.OPLayerNodes = OPLayerNodes
        self.dataFrame = dataFrame
        
        #Input to Hidden Layer Weights with Bias
        self.IPHLWeightList = [[-2.1515666689877735, -1.9123791380660269], [-2.0581216695142124, -1.7557122916725199], [-2.3138601552323506, -2.087370547490641] #self.createRandomWeightListForEachIP(self.inputNodes,self.HLNeurons)
        self.IPHLBiasWeightList = [[-2.4743810393722723], [-2.622221339728828], [-2.251999441299976]] #self.createRandomWeightListForEachIP(1,self.HLNeurons)
        
        #Hidden Layer to Output Layer Weights with Bias
        self.HLOPWeightList = [[-1.4678502038598522, -1.341003660010974, -1.582212131264092], [-1.2185337790930812, -1.2213263874819933, -1.2808752417931764]]#self.createRandomWeightListForEachIP(self.HLNeurons,self.OPLayerNodes)
        self.HLOPBiasWeightList = [[-0.0880397669809113], [0.5560004548630116]]#self.createRandomWeightListForEachIP(1,self.OPLayerNodes)

        self.lambdaValue = lambdaValue            
        self.learningRate = learningRate
        self.momentumRate = momentumRate
        
        self.lastDeltaWtsHLOP = []
        self.lastDeltaBiasWtsHLOP = []
        self.lastDeltaWtsIPHL = []
        self.lastDeltaBiasWtsIPHL = []
        self.allAvgRsmeValues=[]
       
    def getInputValuesFromDataFrame(self,dataFrame,numberOfInputs):
        result=[]
        for i in range(len(newDataFrame)):
            result.append([[xDistanceToTargetNormalised[i]],[yDistanceToTargetNormalised[i]]])
        return result
        
    def createList(self,rows,colums):       # eg: [[1],[1]]
        result=[]
        for i in range(rows):
            row=[]
            for j in range(colums):
                row.append(1)
            result.append(row)
        return result

    def getOutPutValuesFromDataFrame(self,dataFrame,numberOfOutputs):
        result=[]
        for i in range(len(newDataFrame)):
            result.append([[velocityXNormalised[i]],[velocityYNormalised[i]]])
        return result
        
    
    #****TESTING SECTION*****
    
    def predictOutputWithGoodWeights(self,singleInputValue):
         #********CALCULATE FEEDFORWARD***********
        #Getting Hidden Layers Output 
        HLNeuronsOP = self.getActivatedNeuronValues(self.HLNeurons,singleInputValue,self.IPHLWeightList,self.IPHLBiasWeightList)

        #Getting Output Layers Output
        ActualOPNeuronsVal = self.getActivatedNeuronValues(self.OPLayerNodes,HLNeuronsOP,self.HLOPWeightList,self.HLOPBiasWeightList)
        
        return ActualOPNeuronsVal;
        
    #****ACTUAL TRAINING SECTION ******
    
    def computeTrainingOfAllInputs(self,inputValue,desiredOP):
       
        #********CALCULATE FEEDFORWARD***********

        #Getting Hidden Layers Output 
        HLNeuronsOP = self.getActivatedNeuronValues(self.HLNeurons,inputValue,self.IPHLWeightList,self.IPHLBiasWeightList)

        #Getting Output Layers Output
        ActualOPNeuronsVal = self.getActivatedNeuronValues(self.OPLayerNodes,HLNeuronsOP,self.HLOPWeightList,self.HLOPBiasWeightList)

        #Calculating the error between desired OP and actual OP

        errorDesiredOPActualOP = self.calcluateErrorBetDesiredAndActualOP(desiredOP,ActualOPNeuronsVal)

#         deNormalisedOutputs = self.deNormaliseDataSet(desiredOP,self.OPLayerNodes) 

        # ****** CALCULATE BACKPROPOGATION*********

        localOPHLGradientsVal = self.getLocalOutputToHLGradient(ActualOPNeuronsVal,errorDesiredOPActualOP)
        deltaWeightsListHLOP = self.getDeltaWeights(localOPHLGradientsVal,self.lastDeltaWtsHLOP,HLNeuronsOP,self.HLNeurons,self.OPLayerNodes)
        deltaWeightsBiasHLOP = self.getDeltaWeightsForBias(localOPHLGradientsVal,self.lastDeltaBiasWtsHLOP,1,self.OPLayerNodes)


        hogOPHLVal = self.calclulateHOG(HLNeuronsOP,localOPHLGradientsVal,self.HLOPWeightList)
        deltaWtListHLIP = self.getDeltaWeights(hogOPHLVal,self.lastDeltaWtsIPHL,inputValue,self.inputNodes,self.HLNeurons)
        deltaWtBiasHLIP = self.getDeltaWeightsForBias(hogOPHLVal,self.lastDeltaBiasWtsIPHL,1,self.HLNeurons)

         #********ADJUSTING WEIGHTS **********

        adjustedIPHLWts = self.getAdjustedWeights(deltaWtListHLIP,self.IPHLWeightList)
        adjustedIPHLBiasWts = self.getAdjustedWeights(deltaWtBiasHLIP,self.IPHLBiasWeightList)
        adjustedHLOPWts = self.getAdjustedWeights(deltaWeightsListHLOP,self.HLOPWeightList)
        adjustedHLOPBiasWts = self.getAdjustedWeights(deltaWeightsBiasHLOP,self.HLOPBiasWeightList)

#          #*********RMSE CALCULATE*************
#         self.calculateRMSEAtEveryEpoch(errorDesiredOPActualOP)

        #*********UPDATING WEIGHTS ***********

        self.IPHLWeightList = adjustedIPHLWts
        self.IPHLBiasWeightList = adjustedIPHLBiasWts
        self.HLOPWeightList = adjustedHLOPWts
        self.HLOPBiasWeightList = adjustedHLOPBiasWts



        self.lastDeltaWtsHLOP =  deltaWeightsListHLOP
        self.lastDeltaBiasWtsHLOP = deltaWeightsBiasHLOP
        self.lastDeltaWtsIPHL = deltaWtListHLIP
        self.lastDeltaBiasWtsIPHL = deltaWtBiasHLIP
    
    
    def createRandomWeightListForEachIP(self,input_nodes,output_nodes):   # [[],[]]
        result=[]
        for i in range(output_nodes):
            row=[]
            for j in range(input_nodes):
                row.append(round(random.random(),2))
            result.append(row)
        return result

    def sigmoidfun(self,x):
        return 1/(1+(math.exp(-x *self.lambdaValue)))
    
    
    def getActivatedNeuronValues(self,outputNodes,inputsVal,totalWts,biasWTS):
        result=[]
        for i in range(len(totalWts)):
            row=[]
            sum =0
            for a in range(len(inputsVal)):
                sum+=inputsVal[a][0] * totalWts[i][a]
            row.append(self.sigmoidfun(sum+biasWTS[i][0]))
            result.append(row)
        return result

    #------------CALCULATE ERROR BETWEEN DESIRE AND ACTUAL OUTPUTS-----------------
    
    def calcluateErrorBetDesiredAndActualOP(self,desiredOP,actual):
        result=[]
        if len(desiredOP[0]) == len(actual[0]):
            for i in range(len(desiredOP)):
                row=[]
                for j in range(len(desiredOP[i])):
                    row.append(desiredOP[i][j]-actual[i][j])
                result.append(row)
            return result
        else: print("Output length not same")
    
    
    #------------CALCULATE LOCAL GRADIENTS-----------------
    
    def getLocalOutputToHLGradient(self,actualOPValues,errorVals):
        if len(actualOPValues[0]) == len(errorVals[0]):
            result =[]
            for i in range(len(actualOPValues)):
                row =[]
                for j in range(len(actualOPValues[i])):
                    val = self.calLocalOPHLGradient(actualOPValues[i][j],errorVals[i][j])
                    row.append(val)
                result.append(row)
            return result
        
        
    def calLocalOPHLGradient(self,activeOPVal,errorVal):
        return self.lambdaValue * activeOPVal *(1-activeOPVal)*errorVal
    
    
    # --------- CALCULATE DELTA WEIGHTS  -----------

    def getDeltaWeights(self,localOPGradientList,lastDeltaWeights,HLNeuronsOPList,outputNodes,inputNodes):
        if (len(lastDeltaWeights)==0):
            result = self.createList(inputNodes,outputNodes)     # [[1,1]],[1,1]]
            for i in range(len(localOPGradientList)):
                for j in range(len(localOPGradientList[i])):
                    for k in range(outputNodes):
                        fv = self.learningRate* HLNeuronsOPList[k][0] * localOPGradientList[i][j]
                        result[i][k] = fv
            return result
        elif (len(lastDeltaWeights)>0):
            result = self.createList(inputNodes,outputNodes)     # [[1,1]],[1,1]]
            for i in range(len(localOPGradientList)):
                for j in range(len(localOPGradientList[i])):
                    for k in range(outputNodes):
                        fv = self.learningRate* HLNeuronsOPList[k][0] * localOPGradientList[i][j] +(self.momentumRate*lastDeltaWeights[i][k])
                        result[i][k] = fv
            return result
        else: "Please check conditions"
    
    
    def getDeltaWeightsForBias(self,localOPGradientList,lastDeltaWeights,outputNodes,inputNodes):
        if len(lastDeltaWeights)==0:
            result = self.createList(inputNodes,outputNodes)     # [[1,1]],[1,1]]
            for i in range(len(localOPGradientList)):
                for j in range(len(localOPGradientList[i])):
                    for k in range(outputNodes):
                        fv = self.learningRate* 1 * localOPGradientList[i][j]
                        result[i][k] = fv
            return result
        elif len(lastDeltaWeights)>0:
            result = self.createList(inputNodes,outputNodes)     # [[1,1]],[1,1]]
            for i in range(len(localOPGradientList)):
                for j in range(len(localOPGradientList[i])):
                    for k in range(outputNodes):
                        fv = self.learningRate* 1 * localOPGradientList[i][j] +(self.momentumRate*lastDeltaWeights[i][k])
                        result[i][k] = fv
            return result
        else: return "Please check conditions"
    
    #---------CALCULATE HIDDEN OUTPUT GRADIENTS ----------
    
    def calclulateHOG(self,HLNeuronsOPList,localOPGradientList,weightsList):
        result= self.createList(len(HLNeuronsOPList),1)
        for i in range(len(HLNeuronsOPList)):
            v = self.lambdaValue * (HLNeuronsOPList[i][0])*(1-HLNeuronsOPList[i][0]) * self.calculateWeightsSum(weightsList,localOPGradientList,i)
            result[i][0]=v
        return result
    
            
    def calculateWeightsSum(self,weightsList,localOPGradientList,index):
        result =0
        sum =0
        for i in range(len(localOPGradientList)):
            sum+= localOPGradientList[i][0]*weightsList[i][index]
            result = sum
        return result
    
    #--------ADD DELTA WEIGHTS TO ORIGINAL WEIGHTS-------
    
    def getAdjustedWeights(self,deltaweights,oldWeights):
        if len(deltaweights) == len(oldWeights):
            result=[]
            for i in range(len(deltaweights)):
                row=[]
                for j in range(len(deltaweights[i])):
                    wts= oldWeights[i][j] + deltaweights[i][j]
                    row.append(wts)
                result.append(row)
            return result
        else: return "Error"
    
    
    #--------CALCULATE RMSE VALUES---------------
    
    def calculateRMSEAtEveryEpoch(self,errorDesiredOPActualOP):
        result=0
        sqrtErrors=[]
        sum =0
        for i in range(len(errorDesiredOPActualOP)):
            sum+=((errorDesiredOPActualOP[i][0])**2)
        sqrtErrors.append(math.sqrt(sum))

        for i in range(len(sqrtErrors)):
            result+=sqrtErrors[i]
        
        self.allAvgRsmeValues.append(result/len(sqrtErrors))
#         print("RMSE VALUES----------->",result/len(sqrtErrors))
#         print("RMSE VALUES----------->",sqrtErrors)
    
    #----------DENORMALIZE ACTUAL OP------------------
    
#     def dataset_minmax(self,dataset):
#         minmax = []
#         stats = [[min(column), max(column)] for column in zip(*dataset)]
#         return stats


#     def deNormaliseDataSet(self,normaliseData):    
#         deNormalisedData=[]
#         minMaxSets = self.dataset()
#         for i in range(len(normaliseData)):
#             for j in range(self.OPLayerNodes):
#                 if(len(normaliseData[i]==len(minMaxSets[0])))
#         return deNormalisedData 
#     def calDenormalize


SyntaxError: invalid syntax (<ipython-input-17-843b1c26174d>, line 12)

### Neuron Class /Architecture 
Neuron class with following parameters:
 - Number of inputs: -2
 - Number of Hidden Neuorons in Hidden layers - 3
 - Number of Outputnodes -2
 - divided datasets into 70% training set ,15% validation set and 15% testData sets

In [5]:

# Test your Neural Network Architecture
# NeuronClass(IP_nodes,HL_neurons,op_nodes,lambda , learning rate , momentum rate,list of normalised data)

neuronObj = NeuronClass(2,3,2,0.7,0.6,0.9,newDataFrame)

inputvalues = neuronObj.getInputValuesFromDataFrame(newDataFrame,2)
ouputValues = neuronObj.getOutPutValuesFromDataFrame(newDataFrame,2)

trainingSetLength = int(0.70*len(newDataFrame))
validationSetLength = int(0.15*len(newDataFrame))
testSetLength = int(0.15* len(newDataFrame))



### Training of Input data


In [12]:
#*********TRAINING INPUT DATA********
for e in range(100):                                 #epoch loop
    for i in range(0,trainingSetLength):             #inputs loop
        neuronObj.computeTrainingOfAllInputs(inputvalues[i],ouputValues[i])


### Testing of input data

In [14]:
#********TESTING OF DATA************

test=[]
for i in range(trainingSetLength+1,len(newDataFrame)):
    test.append(neuronObj.predictOutputWithGoodWeights(inputvalues[i]))

print(trainingSetLength)
print(test)

8591
[[[0.5161633687836726], [0.35690686292977153]], [[0.5420371466593747], [0.35992994983996074]], [[0.5686290355160047], [0.3630763616793059]], [[0.5956891762428339], [0.366338887519159]], [[0.6229353543072265], [0.36970826731222756]], [[0.6486635761473573], [0.3729910807453523]], [[0.6727442614070479], [0.3761758461331281]], [[0.6951080748893002], [0.37925302000329103]], [[0.7157380900109629], [0.3822149649345777]], [[0.7346602467400151], [0.3850558673513923]], [[0.7519333954832469], [0.38777161634332585]], [[0.7676398790451482], [0.3903596545082242]], [[0.7818772471479926], [0.3928188110671444]], [[0.794751387967338], [0.3951491262849472]], [[0.8063711286922862], [0.3973516747544834]], [[0.8168442056041942], [0.39942839353703485]], [[0.8262744216373975], [0.40138191962870756]], [[0.8347597780153693], [0.4032154398327796]], [[0.8423913686873269], [0.4049325549155583]], [[0.849252847420502], [0.4065371589316957]], [[0.8554203070470958], [0.4080333338250703]], [[0.8609624417381029], [

In [8]:
sum=0
if len(neuronObj.allAvgRsmeValues)>0:
    for i in range(len(neuronObj.allAvgRsmeValues)):
        sum+= neuronObj.allAvgRsmeValues[i]
    print(sum/len(neuronObj.allAvgRsmeValues))

    

In [73]:
print("IPHLWeightList---->", neuronObj.IPHLWeightList,"IPHLBiasWeightList---->",neuronObj.IPHLBiasWeightList,"HLOPWeightList---->",neuronObj.HLOPWeightList,"HLOPBiasWeightList------>",neuronObj.HLOPBiasWeightList)

IPHLWeightList----> [[-2.1515666689877735, -1.9123791380660269], [-2.0581216695142124, -1.7557122916725199], [-2.3138601552323506, -2.087370547490641]] IPHLBiasWeightList----> [[-2.4743810393722723], [-2.622221339728828], [-2.251999441299976]] HLOPWeightList----> [[-1.4678502038598522, -1.341003660010974, -1.582212131264092], [-1.2185337790930812, -1.2213263874819933, -1.2808752417931764]] HLOPBiasWeightList------> [[-0.0880397669809113], [0.5560004548630116]]


In [74]:
print ("xDistanceToTarget > min--->",min(xDistanceToTarget),"max ->",max(xDistanceToTarget))
print ("yDistanceToTarget > min--->",min(yDistanceToTarget),"max ->",max(yDistanceToTarget))
print ("velocityX > min--->",min(velocityX),"max ->",max(velocityX))
print ("velocityY > min--->",min(velocityY),"max ->",max(velocityY))

xDistanceToTarget > min---> -595.4975993 max -> 546.3208359
yDistanceToTarget > min---> 66.0544504 max -> 777.2
velocityX > min---> -6.7 max -> 8.0
velocityY > min---> -7.803695104 max -> 6.305714014


In [ ]:
#         print("IP values                   ---->",inputValue)         
#         print("IP to HL Weights            ---->", self.IPHLWeightList)
#         print("Bias to HL weights          ---->",self.IPHLBiasWeightList)
#         print("HL to OP Weights            ---->", self.HLOPWeightList)
#         print("Bias to OP weights          ---->",self.HLOPBiasWeightList)
#         print("HL neurons O/P              ---->",HLNeuronsOP)
#         print("Actual O/P                  ---->",ActualOPNeuronsVal)
#         print("Desired  O/P                ---->",desiredOP)               
# #             print("Error calculated            ---->",errorDesiredOPActualOP)

#         print("Local OP Gradient               ---->", localOPHLGradientsVal)

#         print("Delta Weights of HL-OP Neurons     ----> ", deltaWeightsListHLOP)
#         print("Delta Weights of HL-OP Bias        ----> ", deltaWeightsBiasHLOP)
#         print("HOG                                ---->",hogOPHLVal)
#         print("Delta Weights of IP-HL Nodes       ---->",deltaWtListHLIP)
#         print("Delta Weights of IP-HL Bias Nodes  ---->",deltaWtBiasHLIP)
#         print("Adjusted Weights of IP-HL layers         ----->",adjustedIPHLWts)
#         print("Adjusted BIAS Weights of IP-HL layers    ----->",adjustedIPHLBiasWts)
#         print("Adjusted Weights of HL-OP layers       ----->",adjustedHLOPWts)
#         print("Adjusted BIAS Weights of HL-OP layers  ----->",adjustedHLOPBiasWts)
#         print("Last Delta Weights of OP-HL Neurons     ----> ", self.lastDeltaWtsHLOP